In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_api_key as g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("Resources/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kokopo,-4.3520,152.2633,77.23,87,15,9.57,PG,2021-09-25 13:04:12
1,1,Hasaki,35.7333,140.8333,66.60,65,100,17.96,JP,2021-09-25 13:04:12
2,2,Busselton,-33.6500,115.3333,57.04,73,88,12.35,AU,2021-09-25 13:04:13
3,3,Tigil,57.8000,158.6667,40.21,76,0,3.44,RU,2021-09-25 13:04:13
4,4,Rikitea,-23.1203,-134.9692,73.74,74,20,16.13,PF,2021-09-25 13:04:14


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)


In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kokopo,-4.3520,152.2633,77.23,87,15,9.57,PG,2021-09-25 13:04:12
9,9,Le Port,-20.9373,55.2919,75.20,75,91,7.87,RE,2021-09-25 13:04:15
10,10,Port Elizabeth,-33.9180,25.5701,76.10,52,98,11.27,ZA,2021-09-25 13:04:16
14,14,Tamandare,-8.7597,-35.1047,79.43,78,18,11.12,BR,2021-09-25 13:04:18
16,16,Anqing,30.5092,117.0506,84.11,67,100,2.95,CN,2021-09-25 13:04:19
19,19,Atuona,-9.8000,-139.0333,77.95,77,10,17.90,PF,2021-09-25 13:04:20
21,21,Ajdabiya,30.7554,20.2263,81.10,38,0,10.76,LY,2021-09-25 12:59:55
24,24,Dwarka,22.2394,68.9678,81.57,83,11,14.38,IN,2021-09-25 13:04:21
25,25,Kutum,14.2000,24.6667,89.15,15,71,9.71,SD,2021-09-25 13:04:22
29,29,Morondava,-20.2833,44.2833,80.35,62,3,14.09,MG,2021-09-25 13:04:23


In [13]:
preferred_cities_df.count()


City_ID       199
City          199
Lat           199
Lng           199
Max Temp      199
Humidity      199
Cloudiness    199
Wind Speed    199
Country       199
Date          199
dtype: int64

In [14]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kokopo,PG,77.23,-4.3520,152.2633,
9,Le Port,RE,75.20,-20.9373,55.2919,
10,Port Elizabeth,ZA,76.10,-33.9180,25.5701,
14,Tamandare,BR,79.43,-8.7597,-35.1047,
16,Anqing,CN,84.11,30.5092,117.0506,
19,Atuona,PF,77.95,-9.8000,-139.0333,
21,Ajdabiya,LY,81.10,30.7554,20.2263,
24,Dwarka,IN,81.57,22.2394,68.9678,
25,Kutum,SD,89.15,14.2000,24.6667,
29,Morondava,MG,80.35,-20.2833,44.2833,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Kokopo,PG,77.23,-4.3520,152.2633,Kokopo Beach Bungalow Resort
9,Le Port,RE,75.20,-20.9373,55.2919,KORBEY D'OR OUEST
10,Port Elizabeth,ZA,76.10,-33.9180,25.5701,39 On Nile Guest House
14,Tamandare,BR,79.43,-8.7597,-35.1047,Pousada 100 Passos
16,Anqing,CN,84.11,30.5092,117.0506,Hongtian Hotel


In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))